In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [34]:
df = pd.read_csv('../Health_cleaned.csv')
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,cpi,rgov,sgov,inter_year,year_of_birth,total_work_earn,total_pension_earn,total_gov_income,total_income,without_work
0,2010,7,2,0,1,1998-06-15,63,.m,4,1,...,0.934777,7394.6318,0.0,1998,1935,0.0,0.00000,7394.6318,7394.63180,1
1,2010,7,2,0,1,1996-10-15,62,.m,3,1,...,0.891173,7374.7271,0.0,1996,1934,0.0,0.00000,7374.7271,7374.72710,1
2,2010,7,2,0,1,2000-03-15,65,.m,3,1,...,0.967805,7029.8052,0.0,2000,1935,0.0,0.00000,7029.8052,7029.80520,1
3,2010,7,2,0,1,1994-07-23,59,.m,3,1,...,0.852219,4023.5574,0.0,1994,1935,0.0,0.00000,4023.5574,4023.55740,1
4,2010,7,2,0,1,1992-09-18,57,.m,3,1,...,0.811126,4193.8506,0.0,1992,1935,0.0,3053.12332,4193.8506,7246.97392,1


In [69]:
df['riwbegy'] = pd.to_datetime(df['riwbegy'])

In [35]:
def count_jobs(row):
    job1 = not (row['rjweeks'] == '.m' or row['rjweeks'] == '0')
    job2 = not (row['rjweek2'] == '.m' or row['rjweek2'] == '0')

    if job1 and job2:
        return 2
    elif job1 or job2:
        return 1
    return 0


df['n_jobs'] = df.apply(count_jobs, axis=1)

In [36]:
df.sort_values('riwbegy', inplace=True)
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,rgov,sgov,inter_year,year_of_birth,total_work_earn,total_pension_earn,total_gov_income,total_income,without_work,n_jobs
19245,22861040,1,2,0,2,1992-04-01,62,56,5,1,...,3522.8347,0.0000,1992,1930,23485.564002,0.000000,3522.8347,27008.398702,1,0
22222,25154020,1,2,0,2,1992-04-01,56,59,3,1,...,0.0000,0.0000,1992,1936,22853.690494,0.000000,0.0000,22853.690494,1,0
22225,25181010,1,2,0,1,1992-04-06,58,53,3,1,...,0.0000,0.0000,1992,1934,43336.457385,0.000000,0.0000,43336.457385,1,0
1112,11067020,1,2,0,1,1992-04-07,53,50,3,0,...,0.0000,0.0000,1992,1939,71295.462149,16775.402859,0.0000,88070.865007,1,0
19239,22861010,1,1,0,2,1992-04-07,56,62,3,1,...,0.0000,3522.8347,1992,1936,23485.564002,0.000000,3522.8347,27008.398702,1,1


In [37]:
unique_ids = df['hhidpn'].unique()
unique_ids.shape

(31532,)

In [64]:
df.columns

Index(['hhidpn', 'rmstat', 'ragender', 'rahispan', 'raracem', 'riwbegy',
       'ragey_b', 'sagey_b', 'rhltc', 'rhlthlm', 'rhibpe', 'rdiabe', 'rcancre',
       'rlunge', 'rhearte', 'rstroke', 'rpsyche', 'rarthre', 'rhosp',
       'rhspnit', 'oop_spend', 'riearn', 'ripena', 'siearn', 'sipena', 'rcovr',
       'rcovs', 'rlbrf', 'rjphys', 'rjlift', 'rjweeks', 'rjweek2', 'rjcten',
       'index_wave', 'insured_gov', 'uninsured', 'retired', 'collegeplus',
       'year', 'cpi', 'rgov', 'sgov', 'inter_year', 'year_of_birth',
       'total_work_income', 'total_pension_income', 'total_gov_income',
       'total_income', 'without_work', 'n_jobs'],
      dtype='object')

In [90]:
income_columns = ['riearn', 'siearn', 'ripena', 'sipena', 'rgov', 'sgov', 'total_work_income',
        'total_pension_income', 'total_gov_income', 'total_income']

In [162]:
def count_deltas(rows):
    income_deltas = {}

    for col in income_columns:
        income_deltas[f'{col}_delta'] = [np.nan]

    indexes = rows.index

    for i, (_, row) in zip(range(len(indexes)), rows.iterrows()):
        if i == 0:
            continue
        prev_row = rows.loc[indexes[i - 1]]

        valid_interval = True
        if row['riwbegy'].year - prev_row['riwbegy'].year > 2:
            valid_interval = False

        for col in income_columns:
            if not valid_interval or prev_row[col] == 0 or row[col] == 0:
                income_deltas[f'{col}_delta'].append(np.nan)
            else:
                income_deltas[f'{col}_delta'].append(row[col] / prev_row[col] * 100 - 100)

    # print(income_deltas)
    return income_deltas

In [170]:
for uid in unique_ids[1000:]:
    income_deltas = count_deltas(df[df['hhidpn'] == uid])

    for col in income_columns:
        df.loc[df['hhidpn'] == uid, f'{col}_delta'] = income_deltas[f'{col}_delta']

In [171]:
df[df['hhidpn'] == 22861010][income_columns + [f'{col}_delta' for col in income_columns]]

,riearn,siearn,ripena,sipena,rgov,sgov,total_work_income,total_pension_income,total_gov_income,total_income,riearn_delta,siearn_delta,ripena_delta,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta
19239,23485.564002,0.0,0.0,0.0,0.0000,3522.8347,23485.564002,0.0,3522.8347,27008.398702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19236,25280.287153,0.0,0.0,0.0,0.0000,3768.0933,25280.287153,0.0,3768.0933,29048.380453,7.641814,NaN,NaN,NaN,NaN,6.961967,7.641814,NaN,6.961967,7.553138
19241,23157.354808,0.0,0.0,0.0,0.0000,5252.3936,23157.354808,0.0,5252.3936,28409.748408,-8.39758,NaN,NaN,NaN,NaN,39.391283,-8.39758,NaN,39.391283,-2.198512
19235,25473.632823,0.0,0.0,0.0,0.0000,10436.9110,25473.632823,0.0,10436.9110,35910.543823,10.002343,NaN,NaN,NaN,NaN,98.707709,10.002343,NaN,98.707709,26.402189
19238,9841.727323,0.0,0.0,0.0,8843.4951,12653.6490,9841.727323,0.0,21497.1441,31338.871423,-61.365042,NaN,NaN,NaN,NaN,21.239407,-61.365042,NaN,105.972285,-12.730724
19240,19852.633829,0.0,0.0,0.0,8338.1064,7279.2988,19852.633829,0.0,15617.4052,35470.039029,101.718999,NaN,NaN,NaN,-5.714807,-42.47273,101.718999,NaN,-27.351256,13.182248
19237,0.000000,0.0,0.0,0.0,8177.9585,7197.1133,0.000000,0.0,15375.0718,15375.071800,NaN,NaN,NaN,NaN,-1.920675,-1.12903,NaN,NaN,-1.551688,-56.653355
19242,0.000000,0.0,0.0,0.0,8028.0000,29991.3520,0.000000,0.0,38019.3520,38019.352000,NaN,NaN,NaN,NaN,-1.833691,316.713629,NaN,NaN,147.279183,147.279183
19243,0.000000,0.0,0.0,0.0,7955.8916,10566.4180,0.000000,0.0,18522.3096,18522.309600,NaN,NaN,NaN,NaN,-0.898211,-64.768451,NaN,NaN,-51.28189,-51.28189
19234,0.000000,0.0,0.0,0.0,0.0000,8585.2588,0.000000,0.0,8585.2588,8585.258800,NaN,NaN,NaN,NaN,NaN,-18.749582,NaN,NaN,-53.649091,-53.649091


In [ ]:
df.to_csv('../Health_cleaned_income_delta.csv', index=False)